# Visualização dos gráficos

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import json
import geojson
from urllib.request import urlopen

# Carregando os dados CSV

In [ ]:
df_city = pd.read_csv("etl_data/city.csv", index_col=0)
df_state = pd.read_csv("etl_data/state.csv", index_col=0)
df_region = pd.read_csv("etl_data/region.csv", index_col=0)

In [ ]:
brazil_code = 100
with urlopen(f'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-{brazil_code}-mun.json') as response:
    brazil_json = json.loads(response.read())

In [ ]:
with open('br_regions.geojson') as f:
    gj = geojson.load(f)

regions_json = gj
regions_json

In [ ]:
df = select_analysis()

In [ ]:
df_brazil = df_city.loc[df_city.city == 'Aracaju'].groupby('month')['cases'].max()
fig = px.area(df_brazil)
df_brazil.head()
fig.show()

In [ ]:
df_brazil = df_city.loc[df_city.city == 'Aracaju'].groupby('month')['cases per day'].sum()
fig = px.area(df_brazil)
df_brazil.head()
fig.show()

In [ ]:
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson') as response:
    Brazil = json.load(response)

In [ ]:
month_cases = pd.DataFrame(df_state.groupby(['full_state', 'month'])['cases per day'].sum()).reset_index()
month_cases

In [ ]:
from plotly.offline import plot

fig = px.choropleth(month_cases, geojson= Brazil, featureidkey = 'properties.name', 
                            locations='full_state', color='cases per day',
                            color_continuous_scale="Viridis",
                            animation_frame = "month",
                            #center = df_region.drop_duplicates('region')[["lat", "lon"]].mean().to_dict(),
                            range_color = (month_cases['cases per day'].min(), month_cases['cases per day'].max()))
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plot(fig)

fig.show()

In [ ]:
region_cases = pd.DataFrame(df_region.groupby(['region', 'month'])['cases per day'].sum()).reset_index()
region_cases

In [ ]:
from plotly.offline import plot

fig = px.choropleth(region_cases, geojson = regions_json, featureidkey = 'properties.nome', 
                            locations='region', color='cases per day',
                            color_continuous_scale="Viridis",
                            animation_frame = "month",
                            #center = df_region.drop_duplicates('region')[["lat", "lon"]].mean().to_dict(),
                            range_color = (region_cases['cases per day'].min(), region_cases['cases per day'].max()))
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plot(fig)

fig.show()

In [ ]:
geo_dict = {
    'city': {
        'dataframe': df_city,
        'geojson': brazil_json,
        'featureidkey': 'properties.id',
        'locations': 'city_code',
    },
    'state': {
        'dataframe': df_state,
        'geojson': Brazil,
        'featureidkey': 'properties.name', 
        'locations': 'full_state'
    }, 
    'region': {
        'dataframe': df_region,
        'geojson': regions_json,
        'featureidkey': 'properties.nome', 
        'locations': 'region'
    }
}

analysis_features = {
    'cases',
    'deaths',
    'cases/population',
    'deaths/population',
    'cases per day/population',
    'deaths per day/population',
    'cases per day',
    'deaths per day'
}

temporal_unities = {
    'month',
    'semester',
    'year',
    'date'
} 

In [ ]:
def select_analysis(feature, time, data):
    # return pd.DataFrame(df_state.groupby(['full_state', 'month'])['cases per day'].sum()).reset_index()
    x = pd.DataFrame(data['dataframe'].groupby([data['locations'], time])[feature])
    return x

In [ ]:
fig = px.choropleth(df_city, geojson=brazil_json, featureidkey = 'properties.id', locations='city_code', color='cases/population',
                           color_continuous_scale="Viridis",
                           animation_frame = "date",
                            #center = df_region.drop_duplicates('region')[["lat", "lon"]].mean().to_dict(),
                          )
fig.update_geos(fitbounds = "locations", visible = False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
plot(fig)

fig.show()